In [32]:
from selenium import webdriver
import csv

def crawl_data(year):
    
    print("processing :" + str(year))
    
    #setting CSV writter
    #win team = 0 : Team 1 win, Win team = 1 : Team 2 Win
    column_list = ["Date","Team_1_Name","Team_2_Name","Team_1_pitcher_name","Team_2_pitcher_name","Team_1_Lineup","Team_2_Lineup","Team_1_Score","Team_2_Score","Win_team"]
    
    with open("statiz_game_data_with_entry_"+str(year)+"2.csv",'w',-1,newline='') as f:
        w = csv.writer(f)
        w.writerow(column_list)
        
        chromedriver_path = './chromedriver'
        baseUrl = "http://www.statiz.co.kr/boxscore.php?opt=1&sopt=1&date="+str(year)+"-07-22"
        endUrl = "http://www.statiz.co.kr/boxscore.php?opt=1&sopt=1&date="+str(year+1)+"-01-01"
        
        #Turn on Chrome
        driver = webdriver.Chrome(chromedriver_path)
        #Open new Tab
        driver.execute_script('window.open("about:blank", "_blank");')
        
        tabs = driver.window_handles
        
        #use tab 1
        driver.switch_to.window(tabs[0])
        driver.get(baseUrl)
        
        while(driver.current_url != endUrl): #repeat while end
            table = driver.find_element_by_xpath("/html/body/div/div[1]/div/section[2]/div/div[2]/div/div")
            
            if table.text.find("경기가 없습니다.") == -1: #means this page has match!
                matches = table.find_elements_by_class_name("box")
                
                for i in range(len(matches)):
                    write_row_list = []
                    
                    match_tables = matches[i].find_elements_by_tag_name("table")
                    date_box = matches[i].find_element_by_class_name("box-title")
                    date_element = date_box.find_element_by_tag_name("b")
                    
                    row_date = date_element.text
                    print(row_date,end=" ") #show processing day
                    
                    if(matches[i].text.find("우천취소") != -1):
                        print("")
                        continue
                    
                    tds = match_tables[0].find_elements_by_tag_name("td")
                    ths = match_tables[0].find_elements_by_tag_name("th")
                    
                    print(len(ths))
                    
                    row_team1_name = tds[0].text
                    row_team1_score = tds[len(ths)-1].text
                    row_team2_name = tds[len(ths)].text
                    row_team2_score = tds[len(ths)*2-1].text
                    
                    print(row_team1_name,end=" ")
                    print(row_team1_score,end=" ")
                    print(row_team2_name,end=" ")
                    print(row_team2_score)
                    
                    if int(row_team1_score) == int(row_team2_score):
                        continue
                    elif int(row_team1_score) > int(row_team2_score):
                        row_win_team = 0
                    else:
                        row_win_team = 1
                    
                    button_divs = matches[i].find_elements_by_class_name("box-body")
                    statics_buttons = button_divs[2].find_elements_by_tag_name("a")
                    match_URL = statics_buttons[2].get_attribute('href') #"경기로그" URL address
                    
                    #use tab 2!
                    driver.switch_to.window(tabs[1])
                    driver.get(match_URL)
                    
                    row_team1_pitcher_name = driver.find_element_by_xpath("/html/body/div/div[1]/div/section[2]/div/div[2]/div[1]/div[1]/div/div[2]/table/tbody/tr[11]/td[2]/a").text
                    row_team1_lineup = ""
                    
                    for i in range(2, 11):
                        row_team1_lineup += driver.find_element_by_xpath("/html/body/div/div[1]/div/section[2]/div/div[2]/div[1]/div[1]/div/div[2]/table/tbody/tr["+str(i)+"]/td[2]/a").text + " "
                        
                    row_team2_pitcher_name = driver.find_element_by_xpath("/html/body/div/div[1]/div/section[2]/div/div[2]/div[1]/div[2]/div/div[2]/table/tbody/tr[11]/td[2]/a").text
                    
                    row_team2_lineup = ""
                    for i in range(2, 11):
                        row_team2_lineup += driver.find_element_by_xpath("/html/body/div/div[1]/div/section[2]/div/div[2]/div[1]/div[2]/div/div[2]/table/tbody/tr["+str(i)+"]/td[2]/a").text + " "
                    
                    write_row_list.append(row_date)
                    write_row_list.append(row_team1_name)
                    write_row_list.append(row_team2_name)
                    write_row_list.append(row_team1_pitcher_name)
                    write_row_list.append(row_team2_pitcher_name)
                    write_row_list.append(row_team1_lineup)
                    write_row_list.append(row_team2_lineup)
                    write_row_list.append(row_team1_score)
                    write_row_list.append(row_team2_score)
                    write_row_list.append(row_win_team)
                    w.writerow(write_row_list)
                    #use tab 1
                    driver.switch_to.window(tabs[0])
                    
            nextButton = driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div/div/div/div/div/a[3]").click()
            table = driver.find_element_by_xpath("/html/body/div/div[1]/div/section[2]/div/div[2]/div/div")

In [33]:
from selenium import webdriver
import csv

crawl_data(2019)
print("job finished!")

processing :2019
2019-07-26 
2019-07-26 11
NC 4 키움 5
2019-07-26 
2019-07-26 11
한화 8 삼성 9
2019-07-26 11
SK 8 롯데 6
2019-07-27 11
KIA 1 두산 12
2019-07-27 11
NC 1 키움 4
2019-07-27 11
LG 5 KT 2
2019-07-27 11
한화 6 삼성 7
2019-07-27 11
SK 4 롯데 0
2019-07-28 
2019-07-28 11
NC 11 키움 0
2019-07-28 11
LG 10 KT 1
2019-07-28 11
한화 6 삼성 13
2019-07-28 11
SK 3 롯데 0
2019-07-30 11
키움 4 LG 2
2019-07-30 11
KIA 2 SK 0
2019-07-30 11
한화 2 KT 3
2019-07-30 11
롯데 2 삼성 5
2019-07-30 11
두산 9 NC 1
2019-07-31 11
키움 8 LG 0
2019-07-31 11
KIA 6 SK 2
2019-07-31 11
한화 5 KT 2
2019-07-31 11
롯데 8 삼성 4
2019-07-31 11
두산 4 NC 8
2019-08-01 11
키움 5 LG 11
2019-08-01 11
KIA 1 SK 10
2019-08-01 11
한화 3 KT 1
2019-08-01 11
롯데 9 삼성 4
2019-08-01 13
두산 3 NC 2
2019-08-03 11
삼성 10 LG 13
2019-08-03 11
KT 7 키움 1
2019-08-03 11
SK 3 한화 0
2019-08-03 11
NC 1 KIA 5
2019-08-03 11
두산 1 롯데 2
2019-08-04 11
삼성 2 LG 1
2019-08-04 11
KT 5 키움 3
2019-08-04 11
SK 2 한화 8
2019-08-04 11
NC 0 KIA 1
2019-08-04 11
두산 4 롯데 9
2019-08-06 11
한화 3 두산 8
2019-08-06 11
KT 3 SK